In [3]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.utils.data
from torch.utils.data import DataLoader

EPSILON = 0.1       # Max perturbation (for L∞ PGD)
ALPHA = 0.01         # Step size per iteration
ATTACK_ITERATIONS = 40
TARGET_LABEL = 2     # Example target label for the targeted attack

# System/Model parameters
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth"
#IQ_FILE_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/Pluto_10_windows_runs2_3/Pluto_10_2m_run3.iq"
IQ_FILE_PATH = "brandan_pluto/brandanpluto10_2m.iq"
from attempt2 import resnet50_1d  # Directly import from attempt2.py
num_classes = 8  # Change this if your model was trained with a different number of classes

# Initialize the model architecture
model = resnet50_1d(num_classes=num_classes).to(DEVICE)

# Load trained weights
print(f"Loading trained model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()


# In[4]:


class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors

Loading trained model weights from: /home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth


/tmp/ipykernel_3836200/1054781179.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_PATH, map_location=DEVICE)


In [4]:
TRUE_IQ_FILE_PATH = IQ_FILE_PATH
BATCH_SIZE = 16
WINDOW_SIZE = 10000
HOP_SIZE = 500
START_INDEX = 4800
END_INDEX = 6000

# Initialize and load model
model = resnet50_1d(num_classes=8)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()


fname = os.path.basename(TRUE_IQ_FILE_PATH)
print(f"\nProcessing file: {TRUE_IQ_FILE_PATH}")

# Extract label from filename: Pluto_#_...
try:
    target_label = int(fname.split("_")[1])
except (IndexError, ValueError):
    print(f"  Warning: could not parse label from filename: {fname}")
    target_label = 0

# Load IQ data
data = np.fromfile(TRUE_IQ_FILE_PATH, dtype="float32")
real_part = data[0::2]
imag_part = data[1::2]

test_data_tensors = []
test_label_tensors = []

# Sliding window generation
for x in range(START_INDEX, END_INDEX):
    start = (x + 1) * HOP_SIZE
    end = start + WINDOW_SIZE
    if end > len(real_part): break

    i_window = real_part[start:end]
    q_window = imag_part[start:end]
    combined = np.vstack((i_window, q_window))  # [2, WINDOW_SIZE]
    test_data_tensors.append(combined)
    test_label_tensors.append(target_label)

if not test_data_tensors:
    print(f"Skipping {fname}: not enough valid IQ segments.")
    

# Stack and shuffle
test_data_tensors = np.stack(test_data_tensors, axis=0)
test_label_tensors = np.array(test_label_tensors)
indices = np.random.permutation(len(test_data_tensors))
test_data = test_data_tensors[indices]
test_labels = test_label_tensors[indices]

# Create Dataset + Loader
test_dataset = IQDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Evaluate
correct = 0
total = 0
mismatch_count = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        mismatch_count += (predicted != labels).sum().item()
        correct += (predicted == labels).sum().item()

acc = correct / total * 100
print(f"[{fname}] Accuracy: {acc:.2f}% | Mismatches: {mismatch_count} / {total}")

/tmp/ipykernel_3836200/1631408887.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))



Processing file: brandan_pluto/brandanpluto10_2m.iq


KeyboardInterrupt: 

In [7]:
def targeted_pgd_attack(model, x, y, target_label, eps, alpha, num_iter):

    model.eval()
    
    if isinstance(target_label, int):
        target_label = torch.full_like(y, target_label)
    
    x_adv = x.clone().detach().to(DEVICE)
    x_adv.requires_grad = True

    for _ in range(num_iter):
        outputs = model(x_adv)
        
        loss = -nn.CrossEntropyLoss()(outputs, target_label)

        model.zero_grad()
        loss.backward()

        grad = x_adv.grad.data
        x_adv = x_adv.detach() + alpha * grad.sign()

        x_adv = torch.min(torch.max(x_adv, x - eps), x + eps)

        x_adv.requires_grad = True

    return x_adv.detach()


# In[48]:


def main():
    print(f"Loading data from: {IQ_FILE_PATH}")

    # Use true label from filename (Pluto_0 → class 0)
    label = 10  # This will be 0
    print(f"True label: {label}, Target label: {TARGET_LABEL}")

    # Load and format IQ data
    data = np.fromfile(IQ_FILE_PATH, dtype="float32")
    real = data[0::2]
    imag = data[1::2]

    start = (START_INDEX + 1) * HOP_SIZE
    end = start + WINDOW_SIZE
    i_window = real[start:end]
    q_window = imag[start:end]
    combined = np.vstack((i_window, q_window))  # [2, N]

    # Wrap in Dataset for normalization
    test_dataset = IQDataset([combined], [label])
    data_tensor, label_tensor = test_dataset[0]
    data_tensor = data_tensor.unsqueeze(0).to(DEVICE)
    label_tensor = label_tensor.unsqueeze(0).to(DEVICE)

    # Prediction before attack
    with torch.no_grad():
        logits = model(data_tensor)
        probs = F.softmax(logits, dim=1)
        orig_pred = torch.argmax(probs, dim=1)
        orig_conf = probs.max(dim=1).values

    print(f"Original prediction: {orig_pred.item()}, Confidence: {orig_conf.item():.4f}")

    # PGD attack targeting label 1
    x_adv = targeted_pgd_attack(
        model=model,
        x=data_tensor,
        y=label_tensor,
        target_label=TARGET_LABEL,
        eps=EPSILON,
        alpha=ALPHA,
        num_iter=ATTACK_ITERATIONS
    )

    with torch.no_grad():
        logits_adv = model(x_adv)
        probs_adv = F.softmax(logits_adv, dim=1)
        adv_pred = torch.argmax(probs_adv, dim=1)
        adv_conf = probs_adv.max(dim=1).values

    print(f"Adversarial prediction: {adv_pred.item()}, Confidence: {adv_conf.item():.4f}")

    # Save perturbation
    original_np = data_tensor.squeeze().cpu().numpy()
    adv_np = x_adv.squeeze().cpu().numpy()
    I_diff = adv_np[0] - original_np[0]
    Q_diff = adv_np[1] - original_np[1]

    interleaved = np.empty(I_diff.size + Q_diff.size, dtype=np.float32)
    interleaved[0::2] = I_diff
    interleaved[1::2] = Q_diff

    save_path = "May19/pluto10_to_target1_noise.iq"
    interleaved.tofile(save_path)
    print(f"Saved perturbation to {save_path}")

main()


Loading data from: brandan_pluto/brandanpluto10_2m.iq
True label: 10, Target label: 2
Original prediction: 7, Confidence: 0.6632
Adversarial prediction: 2, Confidence: 1.0000
Saved perturbation to May19/pluto10_to_target1_noise.iq
